In [3]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 29.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

In [9]:
file_path = '/work/code_embeddings.csv'
data = pd.read_csv(file_path)

data['python_tokens'] = data['python_tokens'].apply(eval)
data['java_tokens'] = data['java_tokens'].apply(eval)

In [13]:
X = np.array([np.abs(np.array(python) - np.array(java)) for python, java in zip(data['python_tokens'], data['java_tokens'])])
y = data['Is_Equal'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [15]:
svm_model = SVC()

In [23]:
param_grid = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': [0.01, 0.1, 1, 10],
    'gamma': [0.01, 'scale', 'auto'],
    'degree': [2, 3, 4]
}

grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='accuracy', cv=10, verbose=1, n_jobs=-1)
grid_search.fit(X_train_resampled, y_train_resampled)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


In [25]:
y_train_pred = best_model.predict(X_train_resampled)
y_test_pred = best_model.predict(X_test)

In [27]:
train_accuracy = accuracy_score(y_train_resampled, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Best Parameters: {best_params}")
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Best Parameters: {'C': 1, 'degree': 4, 'gamma': 'scale', 'kernel': 'poly'}
Train Accuracy: 63.43%
Test Accuracy: 54.86%


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 1, 'degree': 4, 'gamma': 'scale', 'kernel': 'poly'}
Train Accuracy: 63.43%
Test Accuracy: 54.86%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2fe094d0-9fb3-4318-be9c-48cc4ca88a2c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>